In [1]:
#%matplotlib inline
import os
import numpy as np
from PIL import Image
import matplotlib as mp
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
from __future__ import division
import seaborn as sns
sns.set_style('white')
import string
import pandas as pd

import json
import pymongo as pm

## purpose

* upload sketches to S3 [maybe do this later]
* build stimulus dictionary and write to database

## upload sketches to S3 [todo later]

## build stimulus dictionary

In [2]:
## read in experimental metadata file (CSV)
path_to_metadata = 'sketchpad_basic_group_data_for_annotation.csv'
meta = pd.read_csv(path_to_metadata)    

In [3]:
## add parts list
parts =[]
for i in range(meta.shape[0]-1):
    if meta.category[i]=="chair":
        parts.append(["leg,armrest,backrest,seat,pedestal,wheel"])        
    if meta.category[i]=="dog":
        parts.append(["eye,mouth,ear,body,tail,neck,leg"])
    if meta.category[i] == "bird":
        parts.append(["beak,wings,feet,tail,body,head"])
    if meta.category[i] == "car":
        parts.append(["wheel,windshield,body,window,door,bumper,lamp"])
meta = meta.assign(parts=pd.Series(parts))

In [4]:
## add iteration name information
_iterationName = 'sketchpad_basic'
iterationName = [_iterationName]*len(meta)
meta = meta.assign(iterationName=pd.Series(iterationName))

In [5]:
## svg string formatting
svg = []
for i,d in meta.iterrows():    
    splitted = d['svg'].split("'") ## parse string to re-split up into strokes
    svgString = [i for i in splitted if i[0]=='M'] ## check to make sure it is a real start of a spline
    svg.append(svgString)
meta = meta.assign(svg=pd.Series(svg)) 

In [6]:
## add empty games list
games = [[] for i in np.arange(len(meta))]
meta = meta.assign(games=pd.Series(games))

In [7]:
## how many games worth of data do we have?
print '{} unique games worth of data.'.format(len(np.unique(meta.gameID.values)))
print '{} unique sketches.'.format(len(meta))

## write out metadata to json file

## for example:
stimdict = meta.to_dict(orient='records')
stimdict
import json
with open('annotation_meta.js', 'w') as fout:
     json.dump(stimdict, fout)

93 unique games worth of data.
2976 unique sketches.


### upload stim dictionary to mongo (db = 'stimuli', collection='sketchpad_basic_recog')

In [9]:
## load in the JSON that contains the svgData, object labels, and part labels
J = json.loads(open('annotation_meta.js',mode='rU').read())
assert len(J)==len(meta)

In [10]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')

In [11]:
## define the dbname and collection name
db = conn['stimuli']
coll = db['svg_annotation_sketchpad_basic']

In [15]:
## actually add data now to the database (iff reallyRun==True)
reallyRun = True
if reallyRun:
    for (i,j) in enumerate(J):
        if i%250==0:
            print ('%d of %d' % (i,len(J)))
        coll.insert_one(j)
reallyRun = False        

0 of 2976
250 of 2976
500 of 2976
750 of 2976
1000 of 2976
1250 of 2976
1500 of 2976
1750 of 2976
2000 of 2976
2250 of 2976
2500 of 2976
2750 of 2976


In [16]:
## How many sketches do we have in the database?
print 'We have {} sketches.'.format(coll.count())

## What kind of sketches do we have in the database?
print 'We have these kinds: {}'.format(str(coll.distinct('category')))

We have 2976 sketches.
We have these kinds: [u'chair', u'car', u'dog', u'bird']


In [18]:
coll.find_one()

{u'Distractor1': u'crow',
 u'Distractor2': u'redsport',
 u'Distractor3': u'basset',
 u'_id': ObjectId('5b4e70ef8623acc8a2812e07'),
 u'category': u'chair',
 u'condition': u'further',
 u'drawDuration': 18.389,
 u'gameID': u'7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5',
 u'games': [],
 u'iteration': u'pilot2',
 u'iterationName': u'sketchpad_basic',
 u'mean_intensity': 0.0347,
 u'numStrokes': 7,
 u'outcome': True,
 u'parts': [u'leg,armrest,backrest,seat,pedestal,wheel'],
 u'pose': 35,
 u'response': u'knob',
 u'svg': [u'M69,167c0,27.13435 5,54.84972 5,81c0,1.81504 -0.91033,9 1,9',
  u'M160,166c1.05409,0 0.85093,1.9565 1,3c3.93963,27.57739 17,52.46796 17,81c0,1.62072 0.53044,9 2,9',
  u'M59,168c36.94105,0 109.41653,13.58347 137,-14c2.81289,-2.81289 -6,-80.52 -6,-95',
  u'M101,169c0,4.84383 -0.68482,10.20625 0,15c1.75585,12.29095 3,28.12981 3,40c0,1.33333 -0.56667,2 1,2',
  u'M181,164c0,7.86143 13.09325,56.09325 19,62',
  u'M69,163c0,-37.80801 42.62338,-19.06277 67,-15c9.91944,1.65324 31.61601,